# LLM-Langchain

## Init

### Libraries

In [1]:
import datetime
from dotenv import load_dotenv
import os

load_dotenv()

# LLM AWS
import boto3
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_core.tools import tool

# Prompt
from langchain_core.prompts import PromptTemplate

# Output Parser
from langchain_core.output_parsers import JsonOutputParser

### LLMs

In [2]:
## Claude - AWS
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY')
REGION_NAME = os.getenv('REGION_NAME')

boto3.setup_default_session(aws_access_key_id=AWS_ACCESS_KEY,
                            aws_secret_access_key=AWS_SECRET_KEY,
                            region_name=REGION_NAME)

bedrock_client = boto3.client(service_name='bedrock-runtime')

llm_claude = ChatBedrock(
    credentials_profile_name="bedrock-admin",
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=bedrock_client
)

In [ ]:
## GPT4o-mini - Azure


In [ ]:
## Gemini-flash-2 - GCP

### Tools

In [14]:
horas = {1,2,3,19,22}
total = {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,21,22,23,24}

@tool
def agendar_hora(hora: int):
    """
    Recibe una hora para agendar, que va desde 0 a 24 y agenda la hora
    """
    return horas.add(hora)

@tool
def disponibilidad_horaria(hora: int):
    """
    Recibe una hora, que va desde 0 a 24 y consulta en la agenda para ver si está disponible
    """
    if hora in horas:
        return False
    return True

@tool
def listar_horas_disponibles():
    """
    Lista las horas disponibles para agendar
    """
    return total - horas



def agendar_hora_func(hora: int):
    """
    Recibe una hora para agendar, que va desde 0 a 24 y agenda la hora
    """
    return horas.add(hora)


def disponibilidad_horaria_func(hora: int):
    """
    Recibe una hora, que va desde 0 a 24 y consulta en la agenda para ver si está disponible
    """
    if hora in horas:
        return False
    return True


def listar_horas_disponibles_func():
    """
    Lista las horas disponibles para agendar
    """
    return total - horas


In [6]:
tools = [agendar_hora, disponibilidad_horaria, listar_horas_disponibles]
functions = {
    "agendar_hora": agendar_hora,
    "disponibilidad_horaria": disponibilidad_horaria,
    "listar_horas_disponibles": listar_horas_disponibles
}

llm_tools = llm_claude.bind_tools(tools)
llm_tools

RunnableBinding(bound=ChatBedrock(client=<botocore.client.BedrockRuntime object at 0x000001AAE31E2AF0>, credentials_profile_name='bedrock-admin', model_id='anthropic.claude-3-haiku-20240307-v1:0'), kwargs={'tools': [{'name': 'agendar_hora', 'description': 'Recibe una hora para agendar, que va desde 0 a 24 y agenda la hora', 'input_schema': {'properties': {'hora': {'type': 'integer'}}, 'required': ['hora'], 'type': 'object'}}, {'name': 'disponibilidad_horaria', 'description': 'Recibe una hora, que va desde 0 a 24 y consulta en la agenda para ver si está disponible', 'input_schema': {'properties': {'hora': {'type': 'integer'}}, 'required': ['hora'], 'type': 'object'}}, {'name': 'listar_horas_disponibles', 'description': 'Lista las horas disponibles para agendar', 'input_schema': {'properties': {}, 'type': 'object'}}]}, config={}, config_factories=[])

In [19]:
while True:
    horas_sort = list(horas)
    horas_sort.sort()
    print("\033[32m","Horas en uso:",horas_sort)

    print("\033[35mInput:",end="")
    res = input("")
    print(f"\033[35m",res)

    messages = [
    #(
    #    "system",f"La hora actual es: {hora_actual}"
        #"You are a helpful assistant that translates English to French. Translate the user sentence.",
    #),
    ("human", res),
    ]

    ans = llm_tools.invoke(messages)
    #print(ans.content)

    if (ans.tool_calls):
        for call in ans.tool_calls:
            
            function_name = call['name']
            function_args = call['args']

            if function_name == "listar_horas_disponibles":
                print(listar_horas_disponibles_func())
            elif function_name == "disponibilidad_horaria":
                print(disponibilidad_horaria_func(function_args['hora']))
            elif function_name == "agendar_hora":
                agendar_hora(function_args['hora'])
                print(f"agendó una hora '{function_args['hora']}'")
            print()



 Horas en uso: [1, 2, 3, 13, 19, 22]
Input: que horas están disponibles?
{0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 20, 21, 23, 24}

 Horas en uso: [1, 2, 3, 13, 19, 22]
Input: puedo agendar la hora de las 14 o no?
True

 Horas en uso: [1, 2, 3, 13, 19, 22]
Input: y la de las 13?
False

 Horas en uso: [1, 2, 3, 13, 19, 22]
Input: agenda la de las 14
agendó una hora '14'

 Horas en uso: [1, 2, 3, 13, 19, 22]
Input: gracias
 Horas en uso: [1, 2, 3, 13, 19, 22]
Input: que tal
{0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 20, 21, 23, 24}

 Horas en uso: [1, 2, 3, 13, 19, 22]
Input: agenda la hora de las 14
agendó una hora '14'

 Horas en uso: [1, 2, 3, 13, 19, 22]
Input: 


ERROR:root:Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: messages.0: all messages must have non-empty content except for the optional final assistant message


ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: messages.0: all messages must have non-empty content except for the optional final assistant message

### Chain

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.

In [20]:
# Prompt
prompt_related_word = PromptTemplate.from_template("""
Recibirás una lista de palabras, que pueden o no relacionarse entre ellas.
Debes generar una palabra que se relacione con las que te entregué y se pueda incluir en la lista haciendo que mantenga el sentido.

Las palabras son:
{palabras}

La respuesta debes entregármela como un JSON:
respuesta: Es la respuesta que encontraste
justificacion: Justificación corta de por qué elegiste esa palabra
""")

prompt_related_word.format(palabras=["ají","merkén","chile"])

"\nRecibirás una lista de palabras, que pueden o no relacionarse entre ellas.\nDebes generar una palabra que se relacione con las que te entregué y se pueda incluir en la lista haciendo que mantenga el sentido.\n\nLas palabras son:\n['ají', 'merkén', 'chile']\n\nLa respuesta debes entregármela como un JSON:\nrespuesta: Es la respuesta que encontraste\njustificacion: Justificación corta de por qué elegiste esa palabra\n"

In [21]:
# Parser
parser_json = JsonOutputParser()

In [22]:
chain = prompt_related_word | llm_claude | parser_json
chain.invoke({"palabras":["ají","merkén","chile"]})

{'respuesta': 'picante',
 'justificacion': "La palabra 'picante' se relaciona con 'ají', 'merkén' y 'chile', ya que todos estos términos se refieren a ingredientes o alimentos con un sabor intenso y caliente."}

### Agent